<a href="https://colab.research.google.com/github/STTaeKim/fastbook/blob/master/code_10_nlp_ipynb_(Tae_Hoon_Kim_20161993).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 8.5MB/s 
     |████████████████████████████████| 194kB 45.1MB/s 
     |████████████████████████████████| 1.2MB 46.7MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 
Mounted at /content/gdrive


In [ ]:
#hide
from fastbook import *
from IPython.display import display,HTML

[[chapter_nlp]]

# NLP Deep Dive: RNNs

### Word Tokenization with fastai

In [ ]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [ ]:
path.ls()

(#8) [Path('/root/.fastai/data/imdb/models'),Path('/root/.fastai/data/imdb/test'),Path('/root/.fastai/data/imdb/README'),Path('/root/.fastai/data/imdb/train'),Path('/root/.fastai/data/imdb/unsup'),Path('/root/.fastai/data/imdb/tmp_clas'),Path('/root/.fastai/data/imdb/imdb.vocab'),Path('/root/.fastai/data/imdb/tmp_lm')]

In [ ]:
!cp -r /root/.fastai/data/imdb/ /content/sample_data/

In [ ]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [ ]:
files

(#100000) [Path('/root/.fastai/data/imdb/test/pos/1530_10.txt'),Path('/root/.fastai/data/imdb/test/pos/3507_10.txt'),Path('/root/.fastai/data/imdb/test/pos/11308_10.txt'),Path('/root/.fastai/data/imdb/test/pos/6169_8.txt'),Path('/root/.fastai/data/imdb/test/pos/11169_8.txt'),Path('/root/.fastai/data/imdb/test/pos/8451_8.txt'),Path('/root/.fastai/data/imdb/test/pos/10089_9.txt'),Path('/root/.fastai/data/imdb/test/pos/8102_9.txt'),Path('/root/.fastai/data/imdb/test/pos/6383_10.txt'),Path('/root/.fastai/data/imdb/test/pos/5121_8.txt')...]

In [ ]:
files[0]

Path('/root/.fastai/data/imdb/test/pos/1530_10.txt')

In [ ]:
txt = files[0].open().read()

In [ ]:
txt

'What would it be like to be accused of being a subversive? This is what this film explores through the eyes of 2 characters, one being the accused subversive, the other being the interrogator. It is a frightening journey from the beginning to the end. This film is not for everybody and if you do not understand political governments thoroughly, you will never get the point of this film, as proved by 90% of the reviews here.'

In [ ]:
txt[:75]

'What would it be like to be accused of being a subversive? This is what thi'

In [ ]:
# Token is a unit for NLP, and we turn text into a list of token using tokenizer
spacy = WordTokenizer()

In [ ]:
spacy([txt]) #feed the data in a form of a list

<generator object SpacyTokenizer.__call__.<locals>.<genexpr> at 0x7fdf075ddd50>

In [ ]:
toks = first(spacy([txt])) # pick a first element of the tokens
toks

(#86) ['What','would','it','be','like','to','be','accused','of','being'...]

In [ ]:
print(coll_repr(toks, 30))

(#86) ['What','would','it','be','like','to','be','accused','of','being','a','subversive','?','This','is','what','this','film','explores','through','the','eyes','of','2','characters',',','one','being','the','accused'...]


In [ ]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [ ]:
tkn = Tokenizer(spacy) # A wrapping tokenizer, which adds special tokens
print(coll_repr(tkn(txt)), 31)
# xxbos:: Indicates the beginning of a text (here, a review)
# xxmaj:: Indicates the next word begins with a capital (since we lowercased everything)
# xxunk:: Indicates the word is unknown

(#91) ['xxbos','xxmaj','what','would','it','be','like','to','be','accused'...] 31


In [ ]:
defaults.text_proc_rules
# fix_html:: Replaces special HTML characters with a readable version (IMDb reviews have quite a few of these)
# replace_rep:: Replaces any character repeated three times or more with a special token for repetition (xxrep), the number of times it's repeated, then the character
# replace_wrep:: Replaces any word repeated three times or more with a special token for word repetition (xxwrep), the number of times it's repeated, then the word
# spec_add_spaces:: Adds spaces around / and #
# rm_useless_spaces:: Removes all repetitions of the space character
# replace_all_caps:: Lowercases a word written in all caps and adds a special token for all caps (xxup) in front of it
# replace_maj:: Lowercases a capitalized word and adds a special token for capitalized (xxmaj) in front of it
# lowercase:: Lowercases all text and adds a special token at the beginning (xxbos) and/or the end (xxeos)

[<function fastai.text.core.fix_html>,
 <function fastai.text.core.replace_rep>,
 <function fastai.text.core.replace_wrep>,
 <function fastai.text.core.spec_add_spaces>,
 <function fastai.text.core.rm_useless_spaces>,
 <function fastai.text.core.replace_all_caps>,
 <function fastai.text.core.replace_maj>,
 <function fastai.text.core.lowercase>]

In [ ]:
?replace_rep

In [ ]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

### Numericalization with fastai

In [ ]:
txts = L(o.open().read() for o in files[:2000]) # take 2,000 reviews

In [ ]:
toks200 = txts[:200].map(tkn) #tokenizing 200 reviews

In [ ]:
txts[0]

'What would it be like to be accused of being a subversive? This is what this film explores through the eyes of 2 characters, one being the accused subversive, the other being the interrogator. It is a frightening journey from the beginning to the end. This film is not for everybody and if you do not understand political governments thoroughly, you will never get the point of this film, as proved by 90% of the reviews here.'

In [ ]:
toks200[0]

(#91) ['xxbos','xxmaj','what','would','it','be','like','to','be','accused'...]

In [ ]:
num = Numericalize()
num.setup(toks200) # creating a vocab using toks200
coll_repr(num.vocab,20)
# The defaults to Numericalize are min_freq=3,max_vocab=60,000.
# max_vocab=60000 results in fastai replacing all words other than the most common 60,000 with a special unknown word token, xxunk. 
# min_freq=3 means that any word appearing less than three times is replaced with xxunk

"(#2032) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','and','a','of','to','is','in','it','i'...]"

In [ ]:
len(num.vocab)

2032

In [ ]:
toks200

(#200) [['xxbos', 'xxmaj', 'what', 'would', 'it', 'be', 'like', 'to', 'be', 'accused', 'of', 'being', 'a', 'subversive', '?', 'xxmaj', 'this', 'is', 'what', 'this', 'film', 'explores', 'through', 'the', 'eyes', 'of', '2', 'characters', ',', 'one', 'being', 'the', 'accused', 'subversive', ',', 'the', 'other', 'being', 'the', 'interrogator', '.', 'xxmaj', 'it', 'is', 'a', 'frightening', 'journey', 'from', 'the', 'beginning', 'to', 'the', 'end', '.', 'xxmaj', 'this', 'film', 'is', 'not', 'for', 'everybody', 'and', 'if', 'you', 'do', 'not', 'understand', 'political', 'governments', 'thoroughly', ',', 'you', 'will', 'never', 'get', 'the', 'point', 'of', 'this', 'film', ',', 'as', 'proved', 'by', '90', '%', 'of', 'the', 'reviews', 'here', '.'],['xxbos', '"', 'just', 'before', 'dawn', '"', 'is', 'one', 'of', 'the', 'best', 'slasher', 'films.it', 'very', 'realistic', 'and', 'atmospheric.it', 'reminds', 'me', 'xxmaj', 'tobe', 'xxmaj', 'hooper`s', '"', 'the', 'xxmaj', 'texas', 'chainsaw', 'massa

In [ ]:
# After creating Numericalize object, we can use it as if it were a function.
nums = num(toks)[:20]; nums
# This time, our tokens have been converted to a tensor of integers that our model can receive.

TensorText([   0,   83,   18,   42,   55,   15,   42,    0,   14,  116,   13,    0,  108,    0,   16,   72,   20,   29, 1453,  134])

In [ ]:
# We can check that they map back to the original text:
' '.join(num.vocab[o] for o in nums)

'xxunk would it be like to be xxunk of being a xxunk ? xxunk is what this film explores through'

### Putting Our Texts into Batches for a Language Model

In [ ]:
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
splitStream = stream.split()
splitStream, len(stream.split())

(['In',
  'this',
  'chapter,',
  'we',
  'will',
  'go',
  'back',
  'over',
  'the',
  'example',
  'of',
  'classifying',
  'movie',
  'reviews',
  'we',
  'studied',
  'in',
  'chapter',
  '1',
  'and',
  'dig',
  'deeper',
  'under',
  'the',
  'surface.',
  'First',
  'we',
  'will',
  'look',
  'at',
  'the',
  'processing',
  'steps',
  'necessary',
  'to',
  'convert',
  'text',
  'into',
  'numbers',
  'and',
  'how',
  'to',
  'customize',
  'it.',
  'By',
  'doing',
  'this,',
  "we'll",
  'have',
  'another',
  'example',
  'of',
  'the',
  'PreProcessor',
  'used',
  'in',
  'the',
  'data',
  'block',
  'API.',
  'Then',
  'we',
  'will',
  'study',
  'how',
  'we',
  'build',
  'a',
  'language',
  'model',
  'and',
  'train',
  'it',
  'for',
  'a',
  'while.'],
 76)

In [ ]:
tokens = tkn(stream)
tokens

(#90) ['xxbos','xxmaj','in','this','chapter',',','we','will','go','back'...]

In [ ]:
bs,seq_len = 6,15 # breaking the long string of tokens into 6 small segments, and the length of the segment is 15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)]) 

In [ ]:
d_tokens

array([['xxbos', 'xxmaj', 'in', 'this', 'chapter', ',', 'we', 'will', 'go', 'back', 'over', 'the', 'example', 'of', 'classifying'],
       ['movie', 'reviews', 'we', 'studied', 'in', 'chapter', '1', 'and', 'dig', 'deeper', 'under', 'the', 'surface', '.', 'xxmaj'],
       ['first', 'we', 'will', 'look', 'at', 'the', 'processing', 'steps', 'necessary', 'to', 'convert', 'text', 'into', 'numbers', 'and'],
       ['how', 'to', 'customize', 'it', '.', 'xxmaj', 'by', 'doing', 'this', ',', 'we', "'ll", 'have', 'another', 'example'],
       ['of', 'the', 'preprocessor', 'used', 'in', 'the', 'data', 'block', 'xxup', 'api', '.', '\n', 'xxmaj', 'then', 'we'],
       ['will', 'study', 'how', 'we', 'build', 'a', 'language', 'model', 'and', 'train', 'it', 'for', 'a', 'while', '.']], dtype='<U12')

In [ ]:
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


## Training a Text Classifier

### Language Model Using DataBlock

In [ ]:
# fastai handles tokenization and numericalization automatically when TextBlock is passed to DataBlock
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [ ]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj every good thing you 've heard about this movie is true . xxmaj it may very well be the fastest paced movie xxmaj i 've ever seen . xxmaj jerry xxmaj bruckheimer 's most hyperbolic action movie ai n't got nothing ' on this one . \n\n xxmaj cary xxmaj grant and xxmaj rosalind xxmaj russell were a brilliant screen pair ( indeed , it seems that no one was bad casting when paired with xxmaj cary xxmaj","xxmaj every good thing you 've heard about this movie is true . xxmaj it may very well be the fastest paced movie xxmaj i 've ever seen . xxmaj jerry xxmaj bruckheimer 's most hyperbolic action movie ai n't got nothing ' on this one . \n\n xxmaj cary xxmaj grant and xxmaj rosalind xxmaj russell were a brilliant screen pair ( indeed , it seems that no one was bad casting when paired with xxmaj cary xxmaj grant"
1,"cornball romance of xxmaj john and xxmaj isobel seems merely in this plot so that xxmaj bey will screw up endangering his perfect plan which was being carried out successfully before he xxunk it up . xxmaj and , xxmaj bey merely sees her frolicking with xxmaj john on the grass .. the whole "" love - at - first - sight "" rubbish really did n't wash for me . xxmaj plus you have the mummy being able to","romance of xxmaj john and xxmaj isobel seems merely in this plot so that xxmaj bey will screw up endangering his perfect plan which was being carried out successfully before he xxunk it up . xxmaj and , xxmaj bey merely sees her frolicking with xxmaj john on the grass .. the whole "" love - at - first - sight "" rubbish really did n't wash for me . xxmaj plus you have the mummy being able to kill"


### Fine-Tuning the Language Model

In [ ]:
# We create embeddings for the tokens and feed the embeddings to a recurrent neural network (RNN)
# RNN uses an architecture called AWD-LSTM (covered in chapter 12)
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()
# The perplexity metric is often used in language models
# it is the exponential of the loss (i.e., torch.exp(cross_entropy))

In [ ]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.991886,3.901603,0.300608,49.481705,21:11


### Saving and Loading Models

In [ ]:
learn.save('1epoch')
# This will create a file in learn.path/models/ named 1epoch.pth

Path('/root/.fastai/data/imdb/models/1epoch.pth')

In [ ]:
# you can load the content of this file with
learn = learn.load('1epoch')

In [ ]:
# Once the initial training has completed, we can continue fine-tuning the model after unfreezing
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time


KeyboardInterrupt: ignored

In [ ]:
# Once this is done, we save all of our model except the final layer that converts activations to probabilities of picking each token in our vocabulary. 
# The model without the final layer is called the encoder. 
# We can save it with save_encoder
learn.save_encoder('finetuned')
# This completes the second stage of the text classification process: fine-tuning the language model. 
# We can now use it to fine-tune a classifier using the IMDb sentiment labels.

### Text Generation:

###Assignment Answer Part 1:

 **"TEXT ="** will be used as an initial data to analyse and generate movie reviews that starts with this sentence. 



In [ ]:
TEXT = "I do not like this movie, because"
N_WORDS = 50
N_SENTENCES = 15
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

### Assignment Answer Part 2: 

With the use of **"[learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]**", it will help predict the movie reviews in the context of the **"TEXT ="**.


In [ ]:
print("\n".join(preds))
# Our model doesn't have any programmed knowledge of the structure of a sentence or grammar rules, 

# yet it has clearly learned a lot about English sentences

i do not like this movie , because i am a huge Eddie Murphy fan . It is very refreshing to see him in this movie , as is his acting is performance of the same caliber . They are better in some of the movies i have seen , and i am not
i do not like this movie , because it was made in 1962 , but i am not sure where it came from . However , i still remember it . It was a really nice movie . The story follows the way M. Night Shyamalan lives and travels around the world
i do not like this movie , because i have seen it several times now . It was filmed in Brazil . It is also very good to see a movie that has been viewed over and over again to be believed . i was forced to obtain this film when i was a
i do not like this movie , because i have to compare it to " the Carpenters " , or Midnight Cowboy . There are also a few good songs , especially Karen 's song in the movie , " a Sunset on Black and White " , but that
i do not like this movie , because i was wondering if it was so bad . The c

**One example:**


 "i do not like this movie , because i am a huge Eddie Murphy fan . It is very refreshing to see him in this movie , as is his acting is performance of the same caliber . They are better in some of the movies i have seen , and i am not"


### Creating the Classifier DataLoaders

In [ ]:
# from language model fine-tuning to classifier fine-tuning
# A classifier, however, predicts some external label—in the case of IMDb, it's the sentiment of a document.
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)
# TextBlock.from_folder no longer has the is_lm=True parameter.
# We pass the vocab we created for the language model fine-tuning.

###Assignment Part 3:

From the analysis above, **"The Classifier DataLoaders"** helps predict if the movie review is **positive** or **negative**. 

In [ ]:
dls_clas.show_batch(max_n=3)

NameError: ignored

In [ ]:
# Let's see with an example, by trying to create a mini-batch containing the first 10 documents. First we'll numericalize them:
nums_samp = toks200[:10].map(num)

In [ ]:
nums_samp.map(len)

(#10) [91,88,118,294,46,194,523,680,213,170]

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

NameError: ignored

In [ ]:
learn = learn.load_encoder('finetuned')

### Fine-Tuning the Classifier

In [ ]:
# The last step is to train with discriminative learning rates and gradual unfreezing
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

In [ ]:
# Then we can unfreeze a bit more, and continue training:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))
# We reached 94.3% accuracy, which was state-of-the-art performance just three years ago